In [43]:
import cv2
import pandas as pd
import os

In [2]:
speakers = {'speaker_000': pd.DataFrame({
    'start': [9.804754, 107.444822, 119.108659, 128.089983],
    'end': [55.424448, 117.733447, 120.755518, 130.517827]}),
    'speaker_001': pd.DataFrame({
    'start': [0.229202, 55.424448, 118.073005, 120.755518, 129.889643],
    'end': [8.904924, 107.037351, 119.668930, 129.159593, 146.816638]})}

In [8]:
session_id = '1'
interview_id = '2'
current_speaker = 'speaker_000'

base_folder = os.path.join('C:/','Users', 'dalex', 'Documentos compartidos', 'ESGI', '5e_annee', 'PA')
base_folder = os.path.join(base_folder,'files', session_id, interview_id)
input_folder = os.path.join(base_folder, 'input')
output_folder = os.path.join(base_folder, 'output')
output_video_folder = os.path.join(output_folder, current_speaker, 'video_parts')

In [9]:
for file in os.listdir(input_folder):
    print(file)

In [44]:
def multiple_split(videofile: str, parts: pd.DataFrame) -> None:
    
    # Open the input video file
    input_video_path = os.path.join(input_folder, videofile)
    cap = cv2.VideoCapture(input_video_path)
    
    # Get video properties
    fps = cap.get(cv2.CAP_PROP_FPS)
    fourcc = cv2.VideoWriter_fourcc(*'XVID')
    size = (int(cap.get(3)), int(cap.get(4)))
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    for i in range(len(parts)):
        start_time = parts['start'][i]
        end_time = parts['end'][i]
        output_path = os.path.join(output_video_folder, 'part_{:05d}.mp4'.format(i))
        print(output_path)
        
        # Calculate frame indices for starting and ending times
        start_frame = int(start_time * fps)
        end_frame = int(end_time * fps)

        # Set starting frame
        cap.set(cv2.CAP_PROP_POS_FRAMES, start_frame)
        
        # Create VideoWriter object
        out = cv2.VideoWriter(output_path, fourcc, fps, size)
        
        # Read frames and write to output video
        frame_count = 0
        while cap.isOpened() and frame_count <= (end_frame - start_frame):
            ret, frame = cap.read()
            if ret:
                # Write the frame to the output video
                out.write(frame)
                frame_count += 1
            else:
                break

    # Release resources
    cap.release()
    out.release()
    
def split_video(speakers):
    filename = 'raw.mp4'
    current_speaker = 'speaker_000'
    for speaker, df in zip(speakers.keys(), speakers.values()):
        if speaker == current_speaker:
            multiple_split(filename, df)


split_video(speakers)